In [2]:
import pandas as pd 
import numpy as np 
from sklearn.utils import shuffle 
from sklearn.feature_extraction.text import CountVectorizer 

# <하이퍼파라미터 및 데이터 정제 코드>
iteration = 1000 # 반복 횟수 설정
learning_rate = 0.1 # 학습률 설정

data = pd.read_csv("C:\weatherReports.csv") # CSV 파일 로드
data = data[['event_narrative', 'event_type']] # 필요한 컬럼 선택
data = data.dropna(axis=0) # 결측치 제거

# <'Thunderstorm Wind'와 'Hail'을 필터링.>
thunder_data = data[data['event_type'] == 'Thunderstorm Wind']
hail_data = data[data['event_type'] == 'Hail']

data = pd.concat([thunder_data, hail_data]) # 두 데이터 병합
data = shuffle(data) # 데이터 셔플

# <라벨링 (Thunderstorm Wind는 0, Hail은 1로 변환)>
data['event_type'].replace('Thunderstorm Wind', 0, inplace = True)
data['event_type'].replace('Hail', 1, inplace = True)

sentences = [] # 문장을 저장할 리스트 생성
for sentence in data['event_narrative']:
    sentences.append(sentence) # 문장을 리스트에 추가
    
cv = CountVectorizer(stop_words = "english") # CountVectorizer 객체 생성, 영어 불용어 제거
cv.fit(sentences) # BoW 설계를 위한 학습

vectors = cv.transform(sentences).toarray() # 문장을 벡터로 변환
label = data['event_type'].to_numpy() # 라벨을 numpy 배열로 변환

# <가설, 정확도 함수, 데이터셋 분리>
train_rate = 0.7 # 훈련 데이터 비율
train = int(0.7 * len(data)) # 훈련 데이터 수 계산
# <훈련 데이터와 테스트 데이터 분리>
train_x, test_x = vectors[:train], vectors[train:]
train_y, test_y = label[:train], label[train:]
theta = np.random.rand(vectors[0].shape[0], 1) # 가중치 초기화

# <가설 함수 정의>
def hypothesis(x):
    x = x.reshape(9302, 1) # 입력 벡터 재구성
    res = (1 / (1 + np.exp(-(np.dot(theta.T, x))))) # 시그모이드 함수 적용
    return res.item()

# <정확도 계산 함수 정의>
def accuracy():
    cnt = 0
    for i in range(len(test_x)):
        res = 1 if hypothesis(test_x[i]) > 0.5 else 0 # 예측 값 계산
        if res == test_y[i]: cnt += 1 # 예측이 정확하면 카운트 증가
    return cnt / len(test_x) # 정확도 계산

# <세타 갱신 코드>
prev_accuracy = 0.0 # 이전 정확도 저장
for i in range(iteration):
    for j in range(theta.shape[0]):
        temp = train_y[i] - hypothesis(train_x[i])
        theta[j] += (learning_rate * temp.item() * train_x[i, j]) # 가중치 업데이트
    if i % 100 == 0 and i > 100: # 100번째 반복마다 정확도 체크
        current_accuracy = accuracy()
        
        if current_accuracy > prev_accuracy: # 이전 정확도보다 높을 때만 출력
            print(f"학습 횟수 {i}, 정인식률: {current_accuracy * 100:.3f}%")
            prev_accuracy = current_accuracy
        else: # 현재 정확도가 이전 정확도보다 낮을 경우, 이전의 최고 정확도를 유지
            prev_accuracy = max(prev_accuracy, current_accuracy)

학습 횟수 200, 정인식률: 85.253%
학습 횟수 300, 정인식률: 89.674%
학습 횟수 400, 정인식률: 92.216%
학습 횟수 500, 정인식률: 93.540%
학습 횟수 600, 정인식률: 94.625%
학습 횟수 700, 정인식률: 94.731%
학습 횟수 800, 정인식률: 94.996%
학습 횟수 900, 정인식률: 95.367%
